# Welcome to the Flux Breakdown
## This is a work in progress - Due to completion by the end of the week

In this demonstration, we take the most basic, but important, building block of deep learning - the perceptron, and build it up to contextualize images. Most of these blocks have brief explainations as to their intention, but if something is confusing, feel free to let me know!

In [1]:
using Flux

## Part One: A Single Perceptron:
Here we will build a single node, and demonstrate the properties of said node. Start with an x and y that we will use as the inputs and expected outputs of our system. Notice the dimentionality of y doesnt correlate with the output of a single perceptron, we will make use of this one later.

In [ ]:
x = [1]
y = [0.6, 0.4]

Using the dense layer constructor, we will create and instance of a single perceptron with one input and one output. This will also be intialized with a sigmoid nonlinearity function (σ)

In [ ]:
single_perceptron_model = Dense(1,1,σ)

We can look through the properties of this model, like its weight and bias terms. This is shown below, we can print out the arrays holding these values

In [ ]:
single_perceptron_model.W

In [ ]:
single_perceptron_model.b

Say we want to see how the model predicts a value on the basis of our input. By using it as a function with the parameter x, we can see how it predicts this value

In [ ]:
single_perceptron_model(x)

Notice that this is no different that making use of the weight and the bias terms in the form of a linear equation alongside the sigmoid function. This is shown by calculating this directly below

In [ ]:
σ.(single_perceptron_model.W*[1] + single_perceptron_model.b)

## Part Two: Multi-Dimentionality:
Here we will build a chain of outputs to the perceptron, and observe the changes in the weights and bias terms that allow us to visualize this

In [ ]:
single_dense_model = Dense(1,2,σ)

The weight and bias arrays are no longer single dimentional. Finally, we can see that perceptrons are built on vector multiplication, and operations like these are optimized when we enter hardware accelerators, GPUs and TPUs

In [ ]:
single_dense_model.W

In [ ]:
single_dense_model.b

Just like before, we can test the input to the model, this should be the same as taking the multiplications of these vectors, which each element operating on the inner product

In [ ]:
single_dense_model(x)

In [ ]:
σ.(single_dense_model.W*x + single_dense_model.b)

## Part Three: Chaining Operations Together:
What happpens when we want to add continuous layers together to produce a model. We quickly observe the use of the Chain function, which is the basis of creating larger systems in flux. We know one option to chaining the output to first perceptron to the second, is by placing the call to run the model inside the other. 

In [ ]:
single_dense_model(single_perceptron_model([1]))

But doing so is pretty inefficient, we can get the same result, while saving lines and saving the new model using Chain. Chain will also be helpful as we begin to introduce new deep learning elements to the party

In [ ]:
small_mlp = Chain(single_perceptron_model,single_dense_model)

In [ ]:
small_mlp(x)

## Part Four: Getting The Right Results:
Our outputs aren't quite what we wanted. How do we quantify this, and how can we adjust the perceptrons we have to get closer to our desired output. To do this, we need to be able to quantify how far we are, and which way to move

In [2]:
using Flux: mse

Lets start by defining loss using a mean squared error operation, which will aggregate the squared error among all of the expected and actual values. Using this, we can find the right direction to move in during training

In [ ]:
loss(x,y) = mse(small_mlp(x),y)

In [ ]:
original_loss = loss(x,y)

Lets define an optimization algorithm we would like to use. We will make use of the gradient decent with a learning rate of 0.01. Of course, we can use other optimizers, mentioned in comments, with more details in the docs found here:

https://fluxml.ai/Flux.jl/v0.8/training/optimisers/#Optimiser-Reference-1

In [ ]:
opt = Descent(0.01)

We can use the train! function to shift our perceptron parameters in the right direction. Notice that after using the training, we are a little closer to the goal, but we aren't quite there, maybe if we do this enough times we can get it, but how will we do that?

In [ ]:
Flux.train!(loss, params(small_mlp), [(x, y)], opt)

In [ ]:
small_mlp(x)

In [ ]:
new_loss = loss(x,y)

In [ ]:
Δ = original_loss - new_loss

Lets introduce epochs. Epochs indicate the repetition in training that we would like to observe. As an example, if I want to train ten times, then I would train for ten epochs. We test this macro below

In [3]:
using Flux: @epochs

In [ ]:
epochs = 10

In [ ]:
@epochs epochs Flux.train!(loss, params(small_mlp), [(x, y)], opt)

In [ ]:
epoch_loss = loss(x,y)

In [ ]:
Δ = new_loss - epoch_loss

## Part Five: Go Big Mode
We have the building blocks, and understanding of the Flux perceptron, and the knowledge to chain together some layers, but using a dataset of 1, and optimizing only for that isn't something you would ever use a NN model for. We explore the dataloader, and training a model to spec in this next section

In [4]:
using Random
using Plots

In [19]:
x = transpose(shuffle!(collect(1:1000)))

1×1000 LinearAlgebra.Transpose{Int64,Array{Int64,1}}:
 761  636  71  749  137  668  396  689  …  712  816  676  85  639  678  667

In [20]:
y = [i+rand(1:100) for i in x]

1×1000 Array{Int64,2}:
 852  666  139  798  228  684  443  723  …  745  859  766  176  652  774  725

In [21]:
plot(x,y,seriestype = :scatter)

In [22]:
x_train = transpose(x[1:800])
y_train = transpose(y[1:800])
x_test = transpose(x[801:1000])
y_test = transpose(y[801:1000])

1×200 LinearAlgebra.Transpose{Int64,Array{Int64,1}}:
 27  819  807  866  281  532  712  615  …  745  859  766  176  652  774  725

In [23]:
using Flux.Data: DataLoader

In [24]:
train_data = DataLoader(x_train, y_train, batchsize=50, shuffle=true)
test_data = DataLoader(x_test, y_test, batchsize=50)

DataLoader{Tuple{LinearAlgebra.Transpose{Int64,Array{Int64,1}},LinearAlgebra.Transpose{Int64,Array{Int64,1}}}}(([24 802 … 678 667], [27 819 … 774 725]), 50, 200, true, 200, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200], false)

In [25]:
large_mlp = Chain(Dense(1,15),Dense(15,20),Dense(20,1,relu))

Chain(Dense(1, 15), Dense(15, 20), Dense(20, 1, relu))

In [26]:
large_mlp([x_test[1]])

1-element Array{Float32,1}:
 0.0

In [27]:
loss(x,y) = mse(large_mlp(x),y)
loss([x_test[1]],[y_test[1]])

729.0f0

In [35]:
opt = Descent(1e-3)

Descent(0.001)

In [33]:
epochs = 10

10

In [36]:
@epochs epochs Flux.train!(loss, params(large_mlp), train_data, opt, cb = () -> @show(loss_all(train_data, large_mlp)))

loss_all(train_data, large_mlp) = 392451.6f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 39245

┌ Info: Epoch 1
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 2
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 3
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 4
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 5
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 6
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 7
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 8
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 9
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136
┌ Info: Epoch 10
└ @ Main C:\Users\Dell\.julia\packages\Flux\q3zeA\src\optimise\train.jl:136


loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.7f0
loss_all(train_data, large_mlp) = 392451.66f0
loss_all(train_data, large_mlp) = 392451.62f0
loss_all(train_data, large_mlp) = 392451.7f0


## Part Six: Putting it All Together - Working with Images
In this final perceptron section, we take an MNIST dataset of image classification for handwrting, and use the building blocks of multilayer perceptrons to contruct identfication of the 9 digits that may exist in each image. Credit to the Flux Model Zoo Github for the original demonstration.

https://github.com/FluxML/model-zoo

In [ ]:
using PyPlot
using CUDA

In [ ]:
xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
xtest, ytest = MLDatasets.MNIST.testdata(Float32)

In [ ]:
ytrain[1]

In [ ]:
matshow(transpose(xtrain[:,:,1]))

In [ ]:
ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

In [ ]:
ytrain[:,1]

In [ ]:
xtrain = Flux.flatten(xtrain)
xtest = Flux.flatten(xtest)

In [ ]:
train_data = DataLoader(xtrain, ytrain, batchsize=1024, shuffle=true)
test_data = DataLoader(xtest, ytest, batchsize=1024)

In [ ]:
img_model = Chain(Dense(28*28, 32, relu), Dense(32, 10))

In [ ]:
img_model(xtrain[:,1])

In [ ]:
findmax(img_model(xtrain[:,1]))

In [ ]:
loss(x,y) = mse(img_model(x),y)
loss(xtrain[:,1],ytrain[:,1])

In [17]:
function loss_all(dataloader, model)
    l = 0
    for (x,y) in dataloader
        l += mse(model(x),y)
    end
    l/length(dataloader)
end

loss_all (generic function with 1 method)

In [ ]:
epochs = 10

In [ ]:
opt = ADAM(3e-4)

In [ ]:
@epochs epochs Flux.train!(loss, params(img_model), train_data, opt,cb = () -> @show(loss_all(train_data, img_model)))

## Part Seven: A Step Above - Using A Convolutional Net
In this final section, we will use the same MNIST ideas discussed in the final perceptron model, and try making the same predictions using the state of the art - a convolutional neural network. I don't go into super detail because I am writing this during finals and im a bit pooped, but this does demonstrate how we can take extremely similar ideas used in the previous portions of the program to create a whole new architechture. This is one of Flux's strengths - its transferabiliy. Once again, inspriation and portions of the code are derived from the Flux Model Zoo github, link below.

https://github.com/FluxML/model-zoo

In [ ]:
using MLDatasets
using Flux: onehotbatch, onecold, logitcrossentropy, throttle

In [ ]:
    xtrain, ytrain = MLDatasets.MNIST.traindata(Float32)
    xtest, ytest = MLDatasets.MNIST.testdata(Float32)

    # Reshape Data in order to flatten each image into a linear array
    xtrain = Flux.flatten(xtrain)
    xtest = Flux.flatten(xtest)

    # One-hot-encode the labels
    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

In [ ]:
xtrain